## 1. Import and initialize Pyspark environment

Import Spark environment using findspark
Initialize Spark environment
Create SparkContext


In [1]:
!ls ./data/input/

metadata  reviews


In [2]:
from os import path

ROOT_DIR = "./"
DATA_DIR = path.join(ROOT_DIR, 'data')
INPUT_DATA_PATH = path.join(DATA_DIR, 'input', 'reviews')
!ls {INPUT_DATA_PATH}

Movies_and_TV.json.gz


In [3]:
spark

## 2. Read data

Read all the fields in the txt file by applying filter and map 


In [ ]:
"""
Read JSON-formatted data file to dataframe
"""

data = spark.read.json(INPUT_DATA_PATH)
data.show(10)

## 3. Data Exploration

In [5]:
"""
Get summary of the data
To make sure there is no data incompliance 
"""

data_used = data.select('asin', 'reviewerID', 'overall').dropna('any')
summary = data_used.describe().show()

+-------+--------------------+--------------------+------------------+
|summary|                asin|          reviewerID|           overall|
+-------+--------------------+--------------------+------------------+
|  count|             8765568|             8765568|           8765568|
|   mean| 5.482090734266104E9|                null|4.2330105704501975|
| stddev|2.0830512994750662E9|                null|1.2214384170933368|
|    min|          0000143502|A00013803RVZPCZKTT9U|               1.0|
|    max|          B01HJF79XO|       AZZZYAYJQSDOJ|               5.0|
+-------+--------------------+--------------------+------------------+



In [7]:
from pyspark.sql.functions import isnan, when, count, col

data_used.select([count(when(isnan(c), c)).alias(c) for c in data_used.columns]).show()

+----+----------+-------+
|asin|reviewerID|overall|
+----+----------+-------+
|   0|         0|      0|
+----+----------+-------+



In [8]:
"""
Apply count() to findout:

    ` Number of movies
    ` Number of users
"""

n_user = data_used.select('reviewerID').distinct().count()

n_movie = data_used.select('asin').distinct().count()

n_user, n_movie

(3826085, 182032)

In [9]:
"""
Apply group by and count to findout:

    ` Number of reviews from each user 
    ` Number of reviews each movie get
"""

nreview_by_user = data_used.groupBy('reviewerID').count()

nreview_for_movie = data_used.groupBy('asin').count()

nreview_by_user.show(), nreview_for_movie.show()

+--------------+-----+
|    reviewerID|count|
+--------------+-----+
| AVIKFXS6MT2YV|    2|
|A3826GI7UHI7SZ|    9|
|A33GM0OUOWK19O|    1|
|A1687MV0PLK74B|   16|
| AB64DUL65WO6O|   10|
|A1FWW47TZ65PNY|    1|
|A3RJ48YJJ3NOII|   18|
|A1M0G9T633G1C3|    5|
|A2QVL8FGY79WWH|    3|
|A20TI7T43DCSRY|    2|
|A343A2TZEZ9Y86|    1|
| A6GMEO3VRY51S|  207|
|A2DNWSXNZBD204|   23|
| AZS14W9Q9XCUQ|    8|
| AX3NVXGCTQ8AN|    1|
| AYMM8AP7UVA8Y|    1|
|A2E130DG40UA2L|   17|
| ASWG8EJHCWPKC|   21|
|A1UFEP3IOALM8V|    1|
|A2BUPLL6RIQBRB|    1|
+--------------+-----+
only showing top 20 rows

+----------+-----+
|      asin|count|
+----------+-----+
|0783218923|  150|
|0783225911|  512|
|6300185117|  155|
|630025545X|   37|
|6301008944|  741|
|6301304977|   30|
|630165191X|  229|
|6301928482|   11|
|6301929810|   40|
|6301959728|  256|
|6302405696|   19|
|6302643635|   45|
|6302676835|  109|
|6302995779|   29|
|6303026451|   30|
|6303079709|  640|
|6303150896|   27|
|6303209785|   14|
|6303223052|   21|
|630

(None, None)

In [10]:
"""
Apply aggregation to findout: 
    ` Who are the most active users?
    ` Wh bat is the most common movies?
"""

num = 10

most_active = nreview_by_user.orderBy('count', ascending=False)

most_common = nreview_for_movie.orderBy('count', ascending=False)

print("Some of the most active users: ", most_active.show())
print("Some of the most common movies: ", most_common.show())


+--------------+-----+
|    reviewerID|count|
+--------------+-----+
| AV6QDP8Q0ONK4| 4254|
|A1GGOC9PVDXW7Z| 2292|
|A328S9RN3U5M68| 2175|
| ABO2ZI2Y5DQ9T| 2136|
| AWG2O9C42XW5G| 2046|
|A3MV1KKHX51FYT| 2017|
|A2EDZH51XHFA9B| 1907|
|A16CZRQL23NOIW| 1850|
|A3LZGLA88K0LA0| 1826|
|  AIMR915K4YCN| 1763|
|A2NJO6YE954DBH| 1730|
| ANCOMAI0I7LVG| 1667|
|A20EEWWSFMZ1PN| 1666|
|A1D2C0WDCSHUWZ| 1613|
|A2HVL790PBWYTU| 1529|
|A2YUA3H1LLU53Z| 1510|
|A29TKSIWA3JKF3| 1411|
|A19ZXK9HHVRV1X| 1336|
|A2A7NHE5HTK79N| 1262|
|A1NSDP9YZXLMDX| 1208|
+--------------+-----+
only showing top 20 rows

('Some of the most active users: ', None)
+----------+-----+
|      asin|count|
+----------+-----+
|B00YSG2ZPA|24558|
|B00006CXSS|24489|
|B000WGWQG8|23584|
|B00AQVMZKQ|21015|
|B01BHTSIOC|20889|
|B00NAQ3EOK|16857|
|6305837325|16671|
|B00WNBABVC|15284|
|B017S3OP7A|14795|
|B009934S5M|14486|
|B00FL31UF0|14174|
|B014HDTT84|14158|
|B00OGL6S64|14143|
|B0002ERXC2|14007|
|B00R8GUXPG|13761|
|B00PY4Q9OS|13761|
|B00Q0G2VXM|13741|


## 4. Visualization

## 5. Data transformation and save

We want to build a Recommendation system to suggest movies to users who might interested in based on their historical reviews. Rating score should be used. So we will make some information extracting and transforming job this part.

In [13]:
""" Store the ratings data as zipped csv """

target = os.path.join(DATA_DIR, 'ratings')

data_used.write.json(target, mode='overwrite', compression='gzip')